In [1]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.graph_objects as go
from python_module.pricing_model import BlackScholesModel

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# -------
# INPUTS
# -------
S           = 100
K           = 100
T_min       = 1 
T_max       = 250
sigma       = 0.2
r           = 0
option_type = 'call'

# -------
# DISPLAY PV AND GREEKS FOR GIVEN TIME AND SPOT PRICE FOR SINGLE OPTION
# -------
S_list = np.linspace(80, 120, 100)
T_list = [T_min/250, T_max/250]
results = dict()
for S in tqdm(S_list):
    for T in T_list:
        pricing_results = BlackScholesModel.compute_option(S=S, K=K, T=T, r=r, sigma=sigma, option_type=option_type, compute_greeks=True)
        results[(S, T)] = pricing_results
results_df = pd.DataFrame(results).transpose()
results_df.index.names = ['S', 'T']
results_df = results_df.reset_index()
for field in ['price', 'delta', 'gamma_cash', 'vega', 'theta', 'vanna', 'volga']:
    for T in [T_max, T_min]:
        t = T / 250
        df_temp = results_df[results_df['T'] == t]
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df_temp['S'], y=df_temp[field], mode='lines', name=f'T={T:.0f}'))
        fig.update_layout(title=f'{field.capitalize()} vs S for T={T:.2f}', xaxis_title='S', yaxis_title=field.capitalize())
        fig.show()

100%|██████████| 100/100 [00:00<00:00, 2270.37it/s]


In [ ]:
# -------
# INPUTS
# -------
S            = 100
K1           = 100
K2           = 95
T_min        = 10 
T_max        = 250
sigma        = 0.2
r            = 0
option_type1 = 'call'
option_type2 = 'put'
gamma1       = BlackScholesModel.compute_option(S=S, K=K1, T=T_max, r=r, sigma=sigma, option_type=option_type1, compute_greeks=True)['gamma_cash']
gamma2       = BlackScholesModel.compute_option(S=S, K=K2, T=T_max, r=r, sigma=sigma, option_type=option_type2, compute_greeks=True)['gamma_cash']
weight1      = 1
weight2      = - (gamma1 / gamma2)

# -------
# DISPLAY PV AND GREEKS FOR GIVEN TIME AND SPOT PRICE FOR SKEW STRATEGY
# -------
S_list = np.linspace(80, 120, 100)
T_list = [T_min/250, T_max/250]
results = dict()
for S in tqdm(S_list):
    for T in T_list:
        pricing_results1 = BlackScholesModel.compute_option(S=S, K=K1, T=T, r=r, sigma=sigma, option_type=option_type1, compute_greeks=True)
        pricing_results2 = BlackScholesModel.compute_option(S=S, K=K2, T=T, r=r, sigma=sigma, option_type=option_type2, compute_greeks=True)
        pricing_results = weight1 * pd.Series(pricing_results1) + weight2 * pd.Series(pricing_results2)
        results[(S, T)] = pricing_results.to_dict()
results_df = pd.DataFrame(results).transpose()
results_df.index.names = ['S', 'T']
results_df = results_df.reset_index()
for field in ['price', 'delta', 'gamma_cash', 'vega', 'theta', 'vanna', 'volga']:
    for T in [T_max, T_min]:
        t = T / 250
        df_temp = results_df[results_df['T'] == t]
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df_temp['S'], y=df_temp[field], mode='lines', name=f'T={T:.0f}'))
        fig.update_layout(title=f'{field.capitalize()} vs S for T={T:.2f}', xaxis_title='S', yaxis_title=field.capitalize())
        fig.show()

100%|██████████| 100/100 [00:00<00:00, 815.49it/s]


In [17]:
# -------
# INPUTS
# -------
S            = 100
K1           = 100
K2           = 95
K3           = 105
T_min        = 10 
T_max        = 250
sigma        = 0.2
r            = 0
option_type1 = 'call'
option_type2 = 'put'
option_type3 = 'call'
gamma1       = BlackScholesModel.compute_option(S=S, K=K1, T=T_max, r=r, sigma=sigma, option_type=option_type1, compute_greeks=True)['gamma_cash']
gamma2       = BlackScholesModel.compute_option(S=S, K=K2, T=T_max, r=r, sigma=sigma, option_type=option_type2, compute_greeks=True)['gamma_cash']
gamma3       = BlackScholesModel.compute_option(S=S, K=K3, T=T_max, r=r, sigma=sigma, option_type=option_type3, compute_greeks=True)['gamma_cash']
weight1      = 1
weight2      = - ((gamma1 * 0.5) / gamma2)
weight3      = - ((gamma1 * 0.5) / gamma3)

# -------
# DISPLAY PV AND GREEKS FOR GIVEN TIME AND SPOT PRICE FOR FLY STRATEGY
# -------
S_list = np.linspace(80, 120, 100)
T_list = [T_min/250, T_max/250]
results = dict()
for S in tqdm(S_list):
    for T in T_list:
        pricing_results1 = BlackScholesModel.compute_option(S=S, K=K1, T=T, r=r, sigma=sigma, option_type=option_type1, compute_greeks=True)
        pricing_results2 = BlackScholesModel.compute_option(S=S, K=K2, T=T, r=r, sigma=sigma, option_type=option_type2, compute_greeks=True)
        pricing_results3 = BlackScholesModel.compute_option(S=S, K=K3, T=T, r=r, sigma=sigma, option_type=option_type3, compute_greeks=True)
        pricing_results = weight1 * pd.Series(pricing_results1) + weight2 * pd.Series(pricing_results2) + weight3 * pd.Series(pricing_results3)
        results[(S, T)] = pricing_results.to_dict()
results_df = pd.DataFrame(results).transpose()
results_df.index.names = ['S', 'T']
results_df = results_df.reset_index()
for field in ['price', 'delta', 'gamma_cash', 'vega', 'theta', 'vanna', 'volga']:
    for T in [T_max, T_min]:
        t = T / 250
        df_temp = results_df[results_df['T'] == t]
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df_temp['S'], y=df_temp[field], mode='lines', name=f'T={T:.0f}'))
        fig.update_layout(title=f'{field.capitalize()} vs S for T={T:.2f}', xaxis_title='S', yaxis_title=field.capitalize())
        fig.show()

100%|██████████| 100/100 [00:00<00:00, 582.71it/s]
